# Binary Prediction with Rainfall Dataset

**Goal**: Build a robust classifier to predict daily rainfall using environmental data.

**Approach**: Ensemble of Random Forest, AdaBoost, and Logistic Regression with soft voting.

---

## Part 1: Data Loading & Exploratory Data Analysis (EDA)

This section handles:
- Loading training and test datasets
- Basic statistics and data quality checks
- Visualization of feature distributions
- Correlation analysis
- Class imbalance examination

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

import warnings
warnings.filterwarnings('ignore')

### 1.1 Load Data

In [ ]:
train_df = pd.read_csv('https://storage.googleapis.com/kagglesdsdata/competitions/91714/11251744/train.csv?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1764276230&Signature=VPDP9kOA7bV9QqXOUp2HE85r5JoTvn6ZKQoLBYSpok1PGWrtCx15nSB7s1eniVhg0vypnZEIy1FT1AHelVsMFjB9wvPyke8o05heYkXkWcQzt%2BG9BJeergycy3X%2B2ENqy5x2lKhlv1r2TiLVjt7LPqRd%2FYUzCewmrCNxYs%2BsyXIdacWZRfyvMcpW4tgXaOin6QBwDpbZdQLEsoQ%2BDleO%2BvZsl1kEYUA159pdzCDY%2BEOA81f20j7jZsGIdJR3k%2FB5mTvqF3EF8%2BrIf4px5eCzjKg4AOmXEeDXpFYCC4s0dGWIpWVlohG95wRfqSw7c9d4ElWGjT1IYI%2BUSG0syaNGLA%3D%3D&response-content-disposition=attachment%3B+filename%3Dtrain.csv')
test_df = pd.read_csv('https://storage.googleapis.com/kagglesdsdata/competitions/91714/11251744/test.csv?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1764277212&Signature=iV0LMt8woV4SFYatlFCdIQfBKy6fgQ94AFdZVfowDy6uCcEp9fhnkd4PbxB7pRk0yzACnPAudyck5ErHoUS%2FepnqS8LKliBkij%2FpP7zJ2%2FDUf0b6hWNcV20LvCyEQzB3ErZkc8ECMyEcgrFo9w4H8Q%2Bdklx0R5w9bqCNw70UXvtOIPLC8jiLj%2Fz7U%2B5JnfwKH8SLVGutNLKcUiMnPt0QfZBW7TLrYAIklnHZB3JJqePNQNwoAV0Iywg6lopvO8YPHuoaMq1xO5Oi3OKSLGks9QW109vITvrZKSdYA7noomvty27uLEgiFuGCyOK9LqysySTYn7EOF%2FDp3uMJoWidKQ%3D%3D&response-content-disposition=attachment%3B+filename%3Dtest.csv')

print(f"Training data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")
print(f"\nTraining data has {train_df.shape[0]} samples with {train_df.shape[1]} features")
print(f"Test data has {test_df.shape[0]} samples with {test_df.shape[1] - 1} features (no target)")

### 1.2 Initial Data Inspection

In [ ]:
print("First 5 rows of training data:")
display(train_df.head())

print("\nFirst 5 rows of test data:")
display(test_df.head())

In [ ]:
print("Training data info:")
train_df.info()

print("\n" + "="*50)
print("\nTest data info:")
test_df.info()

In [ ]:
print("Missing values in training data:")
missing_train = train_df.isnull().sum()
print(missing_train)
print(f"\nTotal missing values: {missing_train.sum()}")

print("\n" + "="*50)
print("\nMissing values in test data:")
missing_test = test_df.isnull().sum()
print(missing_test)
print(f"\nTotal missing values: {missing_test.sum()}")

### 1.3 Statistical Summary

In [ ]:
print("Statistical Summary of Training Data:")
display(train_df.describe())

### 1.4 Class Distribution & Imbalance Analysis

In [ ]:
# check target variable distribution
rainfall_counts = train_df['rainfall'].value_counts()
rainfall_pcts = train_df['rainfall'].value_counts(normalize=True) * 100

print("Rainfall Class Distribution:")
print(rainfall_counts)
print(f"\nClass Percentages:")
print(f"Rain (1): {rainfall_pcts[1]:.2f}%")
print(f"No Rain (0): {rainfall_pcts[0]:.2f}%")
print(f"\nImbalance Ratio: {rainfall_counts[1] / rainfall_counts[0]:.2f}:1")
print("\n Significant class imbalance found. Will need to address with class_weight='balanced.'")

In [ ]:
# visualize class distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# count plot
sns.countplot(data=train_df, x='rainfall', ax=axes[0], palette='Set2')
axes[0].set_title('Class Distribution (Count)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Rainfall (0=No, 1=Yes)', fontsize=12)
axes[0].set_ylabel('Count', fontsize=12)
axes[0].set_xticklabels(['No Rain', 'Rain'])

# add count labels on bars
for container in axes[0].containers:
    axes[0].bar_label(container)

# pie chart
axes[1].pie(rainfall_counts, labels=['No Rain', 'Rain'], autopct='%1.1f%%', 
            startangle=90, colors=sns.color_palette('Set2'))
axes[1].set_title('Class Distribution (Percentage)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

### 1.5 Feature Distribution Analysis

In [ ]:
# exclude id, day, rainfall
feature_cols = ['pressure', 'maxtemp', 'temparature', 'mintemp', 'dewpoint', 
                'humidity', 'cloud', 'sunshine', 'winddirection', 'windspeed']

# plot distributions
fig, axes = plt.subplots(5, 2, figsize=(15, 18))
axes = axes.ravel()

for idx, col in enumerate(feature_cols):
    axes[idx].hist(train_df[col], bins=30, color='skyblue', edgecolor='black', alpha=0.7)
    axes[idx].set_title(f'Distribution of {col}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel(col, fontsize=10)
    axes[idx].set_ylabel('Frequency', fontsize=10)
    axes[idx].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

### 1.6 Feature Distributions by Rainfall Class

In [ ]:
# box plots comparing feature distributions between rain, no-rain days
fig, axes = plt.subplots(5, 2, figsize=(15, 18))
axes = axes.ravel()

for idx, col in enumerate(feature_cols):
    sns.boxplot(data=train_df, x='rainfall', y=col, ax=axes[idx], palette='Set2')
    axes[idx].set_title(f'{col} by Rainfall', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Rainfall (0=No, 1=Yes)', fontsize=10)
    axes[idx].set_xticklabels(['No Rain', 'Rain'])

plt.tight_layout()
plt.show()

### 1.7 Correlation Analysis

In [ ]:
correlation_matrix = train_df[feature_cols + ['rainfall']].corr()

# visualize correlation heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Heatmap', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

In [ ]:
# features most correlated with rainfall
rainfall_corr = correlation_matrix['rainfall'].drop('rainfall').sort_values(ascending=False)

print("Features correlation with Rainfall (sorted):")
print(rainfall_corr)

# visualize
plt.figure(figsize=(10, 6))
rainfall_corr.plot(kind='barh', color='teal', edgecolor='black')
plt.title('Feature Correlation with Rainfall', fontsize=14, fontweight='bold')
plt.xlabel('Correlation Coefficient', fontsize=12)
plt.ylabel('Features', fontsize=12)
plt.axvline(x=0, color='black', linestyle='--', linewidth=0.8)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

### 1.8 Key Insights from EDA

**Summary of findings**:
- Dataset has 2190 training samples and 730 test samples
- Significant class imbalance: 75.3% rain vs 24.7% no rain
- Features show varying distributions and correlations with rainfall
- Strong multicollinearity expected between temperature variables (maxtemp, temparature, mintemp, dewpoint)

---

## Part 2: Data Preprocessing & Feature Engineering

This section handles:
- Dropping non-predictive features (id, day)
- Feature engineering (creating new features)
- Feature scaling (standardization)
- Train/validation split with stratification

### 2.1 Feature Engineering

In [ ]:
def engineer_features(df):
    """
    Create new features from existing ones.
    
    Args:
        df: DataFrame with raw features
        
    Returns:
        DataFrame with engineered features added
    """
    df = df.copy()
    
    # temperature range (diurnal temperature variation)
    df['temp_range'] = df['maxtemp'] - df['mintemp']
    
    # dewpoint depression (how close air is to saturation)
    # lower values = air closer to saturation = more likely to rain
    df['dewpoint_depression'] = df['temparature'] - df['dewpoint']
    
    # temperature deviation from daily average
    df['temp_from_avg'] = df['temparature'] - (df['maxtemp'] + df['mintemp']) / 2
    
    # interaction: high humidity with low dewpoint depression
    df['humidity_dewpoint_interaction'] = df['humidity'] * (1 / (df['dewpoint_depression'] + 1))
    
    return df

print("Applying feature engineering...")
train_engineered = engineer_features(train_df)
test_engineered = engineer_features(test_df)

print(f"\nOriginal training features: {train_df.shape[1]}")
print(f"After feature engineering: {train_engineered.shape[1]}")
print(f"\nNew features added: {list(set(train_engineered.columns) - set(train_df.columns))}")

In [ ]:
# examine new features
print("Sample of engineered features:")
display(train_engineered[['temp_range', 'dewpoint_depression', 'temp_from_avg', 
                          'humidity_dewpoint_interaction', 'rainfall']].head(10))

# check correlation of new features with rainfall
new_features = ['temp_range', 'dewpoint_depression', 'temp_from_avg', 'humidity_dewpoint_interaction']
new_feature_corr = train_engineered[new_features + ['rainfall']].corr()['rainfall'].drop('rainfall').sort_values(ascending=False)

print("\nNew features correlation with Rainfall:")
print(new_feature_corr)

### 2.2 Handle Missing Values

In [ ]:
# impute missing winddirection value in test data
# use median from training data to avoid data leakage

winddirection_median = train_engineered['winddirection'].median()

print(f"Missing values before imputation:")
print(f"Training set: {train_engineered['winddirection'].isnull().sum()}")
print(f"Test set: {test_engineered['winddirection'].isnull().sum()}")

# apply median imputation to test set
test_engineered['winddirection'].fillna(winddirection_median, inplace=True)

print(f"\nImputation value used: {winddirection_median}")
print(f"\nMissing values after imputation:")
print(f"Training set: {train_engineered['winddirection'].isnull().sum()}")
print(f"Test set: {test_engineered['winddirection'].isnull().sum()}")

### 2.3 Prepare Features and Target

In [ ]:
# non-predictive columns
columns_to_drop = ['id', 'day']

# separate features and target for training data
X = train_engineered.drop(columns=columns_to_drop + ['rainfall'])
y = train_engineered['rainfall']

# prepare test data (no target)
X_test = test_engineered.drop(columns=columns_to_drop)
test_ids = test_engineered['id']

print(f"Training features shape: {X.shape}")
print(f"Training target shape: {y.shape}")
print(f"Test features shape: {X_test.shape}")
print(f"\nFeatures used for modeling:")
print(list(X.columns))

### 2.4 Train/Validation Split (Stratified)

In [ ]:
from sklearn.model_selection import train_test_split

# split with stratification to maintain class distribution
X_train, X_val, y_train, y_val = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Validation set size: {X_val.shape[0]} samples")

# verify stratification maintained class distribution
print(f"\nTraining set class distribution:")
print(y_train.value_counts(normalize=True) * 100)

print(f"\nValidation set class distribution:")
print(y_val.value_counts(normalize=True) * 100)

### 2.5 Feature Scaling (Standardization)

In [ ]:
from sklearn.preprocessing import StandardScaler

# initialize scaler
scaler = StandardScaler()

# fit on training data only, then transform all sets
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# convert back to df
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns, index=X_train.index)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=X.columns, index=X_val.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns, index=X_test.index)

print("Feature scaling completed.")
print(f"\nScaled training features (first 5 rows):")
display(X_train_scaled.head())

# verify scaling: mean ~ 0, std ~ 1
print("\nVerifying standardization (should be ~0 mean, ~1 std):")
print(f"Mean of scaled features:\n{X_train_scaled.mean().round(6)}")
print(f"\nStd of scaled features:\n{X_train_scaled.std().round(6)}")

### 2.6 Final Preprocessed Data Summary

In [ ]:
print("="*60)
print("PREPROCESSING SUMMARY")
print("="*60)

print(f"\n Dataset Splits:")
print(f"  - Training set: {X_train_scaled.shape[0]} samples")
print(f"  - Validation set: {X_val_scaled.shape[0]} samples")
print(f"  - Test set: {X_test_scaled.shape[0]} samples")

print(f"\n Feature Engineering:")
print(f"  - Original features: 10")
print(f"  - Engineered features: 4")
print(f"  - Total features: {X_train_scaled.shape[1]}")

print(f"\n Applied Transformations:")
print(f"  - Dropped non-predictive columns: id, day")
print(f"  - Created engineered features: temp_range, dewpoint_depression, temp_from_avg, humidity_dewpoint_interaction")
print(f"  - Standardized all features (mean=0, std=1)")
print(f"  - Stratified train/validation split (80/20)")

print(f"\n Class Imbalance:")
print(f"  - Rain: 75.3%, No Rain: 24.7%")
print(f"  - Will use class_weight='balanced' in models")

### 3.1 Imports 

In [ ]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

### 3.2 Model Training 

In [ ]:
# Initialize Classifiers-

rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, class_weight='balanced')

ada = AdaBoostClassifier(n_estimators=50, learning_rate=1.0, random_state=42)

logreg = LogisticRegression(random_state=42, class_weight='balanced')


# Train the models

rf.fit(X_train_scaled, y_train)
ada.fit(X_train_scaled, y_train)
logreg.fit(X_train_scaled, y_train);

---

## Next Steps



**Part 4**: Build ensemble classifier with soft voting

**Part 5**: Generate predictions on test set and evaluate performance